## Логистическая регрессия - домашнее задание

По данным тренировочной выборки, постройте модель логистической регрессиии для бинарной классификации.

Используя модель, по тестовой выборке проведите бинарную классификацию объектов тестовой выборки.

Данные каждого варианта находятся в двух файлах.

Переменная `train$output` известный результат классификации в тренировочной выборке, остальные переменные предикторы,
причем среди них до 4 переменных могут быть незначимыми.  Лучше будет это выяснить и убрать их модели

К варианту с номером 0 прилагается файл с ответом, дающим 100% результат.

Постройте модель логистической регрессии по тренировочной выборке. Обратите внимание, предикторы имеют не дискретный характер.


Проведите классификацию объектов тестовой выборки. Постройте файл с результатами классификации в следующем формате. Название колонки с результатом классификации `x` не имеет значения и может быть любым.

В результате смотрите полученную confusion matrix и другие метрики, на основе которых будет засчитываться или не засчитываться задание

#### **Загрузка данных**

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.preprocessing import StandardScaler

train = pd.read_csv('var_10_train.csv')
test = pd.read_csv('var_10_test.csv')

X = train.drop('output', axis=1)
y = train['output']

#### **Отбор значимых признаков**
Посмотрим с помощью statmodels на модель логистической регресии, и на то, как на нее влияют признаки.

In [2]:
import statsmodels.api as sm

In [3]:
X_1 = sm.add_constant(X)
model = sm.OLS(y, X_1).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 output   R-squared:                       0.691
Model:                            OLS   Adj. R-squared:                  0.656
Method:                 Least Squares   F-statistic:                     19.99
Date:                Sun, 06 Apr 2025   Prob (F-statistic):           1.49e-35
Time:                        20:28:36   Log-Likelihood:                -24.122
No. Observations:                 200   AIC:                             90.24
Df Residuals:                     179   BIC:                             159.5
Df Model:                          20                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.5201      0.023     22.585      0.000       0.475       0.566
x1             0.0399      0.021      1.864      0.064      -0.002       0.082
x2            -0.0205      0.019     -1.058      0.292      -0.059       0.018
x3             0.0025      0.019      0.130      0.896      -0.035       0.040
x4             0.0278      0.017      1.617      0.108      -0.006       0.062
x5             0.0087      0.015      0.585      0.560      -0.021       0.038
x6             0.1833      0.014     12.897      0.000       0.155       0.211
x7             0.1215      0.012     10.063      0.000       0.098       0.145
x8             0.0297      0.013      2.324      0.021       0.004       0.055
x9            -0.0172      0.011     -1.610      0.109      -0.038       0.004
x10           -0.0127      0.012     -1.077      0.283      -0.036       0.011
x11            0.0382      0.010      3.676      0.000       0.018       0.059
x12            0.0169      0.010      1.674      0.096      -0.003       0.037
x13            0.0079      0.010      0.763      0.447      -0.013       0.028
x14            0.0338      0.009      3.875      0.000       0.017       0.051
x15           -0.0013      0.009     -0.152      0.879      -0.018       0.016
x16           -0.0012      0.008     -0.156      0.876      -0.017       0.015
x17            0.0536      0.008      6.608      0.000       0.038       0.070
x18            0.0239      0.007      3.234      0.001       0.009       0.038
x19            0.0417      0.008      5.238      0.000       0.026       0.057
x20            0.0452      0.008      6.004      0.000       0.030       0.060
==============================================================================
Omnibus:                       25.382   Durbin-Watson:                   1.728
Prob(Omnibus):                  0.000   Jarque-Bera (JB):                7.294
Skew:                           0.002   Prob(JB):                       0.0261
Kurtosis:                       2.064   Cond. No.                         3.76
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Перечисленные признаки: $x_3, x_5,x_{15}, x_{16}$ можно считать статистически незначимыми и потенциально исключить из модели, так как у них большой уровень P>|t|

In [4]:
X = X.drop(columns=['x3', 'x5', 'x15', 'x16'])

#### **Обучение модели и оценка результатов**

In [5]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X, y)

y_pred = model.predict(X)
print("f1 score = ", f1_score(y_pred, y))

f1 score =  0.9873417721518988


In [6]:
pd.DataFrame(confusion_matrix(y_pred, y))

,0,1
0,80,2
1,1,117


In [7]:
from tabulate import tabulate

report = classification_report(y, y_pred, output_dict=True)
print(tabulate(pd.DataFrame(report).transpose(), headers='keys', tablefmt='psql'))

+--------------+-------------+----------+------------+-----------+
|              |   precision |   recall |   f1-score |   support |
|--------------+-------------+----------+------------+-----------|
| 0            |    0.97561  | 0.987654 |   0.981595 |    81     |
| 1            |    0.991525 | 0.983193 |   0.987342 |   119     |
| accuracy     |    0.985    | 0.985    |   0.985    |     0.985 |
| macro avg    |    0.983568 | 0.985424 |   0.984468 |   200     |
| weighted avg |    0.98508  | 0.985    |   0.985014 |   200     |
+--------------+-------------+----------+------------+-----------+


In [9]:
X_test = test.drop(columns=['x3', 'x5', 'x15', 'x16'])
y_pred_test = model.predict(X_test)
y_pred_test

/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


array([0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0,
       1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1])

In [10]:
results = pd.DataFrame({'predict': y_pred_test})
results.to_csv('test_pred.csv', index=False)

In [11]:
results

,predict
0,0
1,0
2,1
3,1
4,0
...,...
95,1
96,1
97,0
98,1
